In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from textblob import TextBlob
from keras.models import load_model

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from time import time

In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 170000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 160
# This is fixed.
EMBEDDING_DIM = 100

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
train=pd.read_csv('gdrive/My Drive/CZ4034_train.csv')

In [ ]:
def clean_text(text):
  text=contractions.fix(text)
  text=re.sub(r'\W+',' ',text)
  text=re.sub('\s\s+',' ',text)
  text=text.strip()
  text=text.lower()
  return text

In [ ]:
train['text']=train['text'].apply(clean_text)

In [ ]:
train

,Unnamed: 0,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,favoritecount,text,hashtags,sentiment
0,9471,emmyrm1290,â™¡ 18â™¡ â€œEmily Rose of the Emily Roses â™¡,she/her,187,284,13197,2016-02-05 18:25:21+00:00,2022-02-03 01:03:56+00:00,0,4,me and them discussing drinking vaccine liquid...,[],neutral
1,6084,EduCoder,"Father, software developer, former math teache...","Quebec, Canada",911,1943,8551,2012-12-15 15:40:47+00:00,2022-02-03 15:53:00+00:00,0,0,if you are pro vaccine take a quick second to ...,"[{'text': 'spotifyexodus', 'indices': [261, 27...",positive
2,2481,Suzy_NotSuzy,Starting over again AGAIN. Hello all you beaut...,NaN,411,401,291,2022-01-31 18:51:24+00:00,2022-02-04 03:27:15+00:00,8,22,how can you call me an anti vaxxer when i have...,[],negative
3,5128,krae9836,"Christian, Husband, Father, Son, Libertarian. ...","Savannah, GA",321,59,2529,2018-07-28 01:35:52+00:00,2022-02-03 18:52:23+00:00,1,6,just a piece of advice if you are a parent who...,[],neutral
4,7582,TheGirlyCart,nothing to see here,NaN,4,1,3,2021-11-01 02:37:08+00:00,2022-02-03 09:46:35+00:00,2,0,why are we students forced to give offline exa...,"[{'text': 'weWantOnlineExam', 'indices': [241,...",negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7822,919,DlcFredericks,Dream Big!,"Ontario, CA",276,40,347,2011-11-10 03:09:02+00:00,2022-02-04 12:15:17+00:00,0,1,it is hard being a left leaning person against...,"[{'text': 'TruckersForFreedom', 'indices': [18...",negative
7823,4307,FunkyPopsicles,Baseball fan. #LGM Music fan. Believer of God....,"Ohio, USA",652,351,9170,2021-01-27 16:07:33+00:00,2022-02-03 21:49:02+00:00,0,0,after a very long discussion with a rational a...,[],positive
7824,5699,cakebops,Aspiring to be a permanent resident of Kicksvi...,"San Francisco, CA",213,55,2936,2018-08-25 22:14:27+00:00,2022-02-03 17:00:04+00:00,0,0,let joe rogan be a disgusting racist all he wa...,[],negative
7825,537,tobihoyedepo,"Have fun, live life, enjoy every moment ...we ...","Lagos, Nigeria",1041,74,352,2016-10-01 14:58:26+00:00,2022-02-04 13:56:29+00:00,0,1,south africa makes its own version of moderna ...,[],positive


In [ ]:
train['text'].apply(len).mean()

152.2961543375495

In [ ]:
with open('gdrive/My Drive/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
X = tokenizer.texts_to_sequences(train['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
Y = pd.get_dummies(train['sentiment']).values

In [ ]:
train_text, val_text, train_labels, val_labels = train_test_split(X, Y ,
                                                                    random_state=2021, 
                                                                    test_size=0.2, 
                                                                    stratify=Y)

In [ ]:
# create the model
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())  
checkpoint_filepath="gdrive/My Drive/CZ4034_cnnlstm.hdf5"
#file=os.path.dirname(filepath)
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=False)
callbacks_list = [checkpoint]
model.fit(train_text, train_labels, epochs=50, batch_size=100,verbose = 1,callbacks = callbacks_list,validation_data=(val_text,val_labels))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 160, 100)          17000000  
                                                                 
 conv1d_2 (Conv1D)           (None, 160, 32)           9632      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 80, 32)           0         
 1D)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 100)               53200     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 17,063,135
Trainable params: 17,063,135
Non-trainable params: 0
__________________________________________

In [ ]:
model.save('gdrive/My Drive/CZ4034_CNN_LSTM.h5')

In [ ]:
model=load_model('gdrive/My Drive/CZ4034_CNN_LSTM.h5')

In [ ]:
test=pd.read_csv('gdrive/My Drive/CZ4034_test.csv')

In [ ]:
test['text']=test['text'].apply(clean_text)

In [ ]:
with open('gdrive/My Drive/CNN_LSTM_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (1957, 160)


In [ ]:
Y_test = pd.get_dummies(test['sentiment']).values

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
scores

[2.0847225189208984, 0.7432950139045715]

In [ ]:
pred = model.predict(X_test) 

In [ ]:
pred

array([[5.55670204e-06, 9.99993324e-01, 1.12543921e-06],
       [5.19173136e-06, 9.99993682e-01, 1.03890875e-06],
       [3.10667492e-05, 9.99920964e-01, 4.79359769e-05],
       ...,
       [9.75316882e-01, 2.46800371e-02, 3.01525756e-06],
       [1.08539098e-05, 9.99976397e-01, 1.27336325e-05],
       [9.99996662e-01, 2.16720991e-06, 1.16380477e-06]], dtype=float32)

In [ ]:
pred1=np.argmax(pred,axis=1)

In [ ]:
#pred1=[['negative','neutral','positive'][i] for i in pred1]

In [ ]:
true=np.argmax(Y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(true, pred1)

0.7432950191570882

In [ ]:
print(classification_report(true, pred1))

              precision    recall  f1-score   support

           0       0.66      0.69      0.68       448
           1       0.84      0.77      0.80       494
           2       0.74      0.76      0.75       624

    accuracy                           0.74      1566
   macro avg       0.75      0.74      0.74      1566
weighted avg       0.75      0.74      0.74      1566

